[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_html_to_datasets.ipynb)

In [ ]:
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install datasets bs4

In [ ]:
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

In [ ]:
!mkdir plushies

In [ ]:
import urllib.request
from bs4 import BeautifulSoup

with open('/content/plushies.txt') as html:
    content = html.read()

soup = BeautifulSoup(content)
for imgtag in soup.find_all('img'):
  url=imgtag['src']
  name = url.split('/')[-1]
  headers={'user-agent': 'Mozilla/5.0'}
  r=requests.get(url, headers=headers)
  with open(f"/content/plushies/{name}", 'wb') as f:
      f.write(r.content)

In [ ]:
from datasets import load_dataset, Dataset, Image

with open('/content/plushies.txt') as html:
  content = html.read()

texts = []
images = []
soup = BeautifulSoup(content)
for imgtag in soup.find_all('img'):
  texts.append(imgtag['alt'])
  images.append(f"/content/plushies/{imgtag['src'].split('/')[-1]}")
  
ds = Dataset.from_dict({"image": images, "text": texts})
ds = ds.cast_column("image", Image())

In [ ]:
ds.push_to_hub("camenduru/plushies")

In [ ]:
ds = load_dataset("camenduru/plushies")

In [ ]:
ds.shape